In [ ]:
import os
import shutil
from google.colab import files

# 1. Upload API Token
print("--- UPLOAD YOUR kaggle.json FILE NOW ---")
files.upload()

# 2. Setup Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle

# 3. Download The Specific Datasets
print("\n--- Downloading Cocoa Data (zaldyjr) ---")
!kaggle datasets download -d zaldyjr/cacao-diseases
!unzip -q cacao-diseases.zip -d temp_cocoa

print("\n--- Downloading Maize Data (smaranjitghose) ---")
!kaggle datasets download -d smaranjitghose/corn-or-maize-leaf-disease-dataset
!unzip -q corn-or-maize-leaf-disease-dataset.zip -d temp_maize

print("\n Downloads Complete.")

--- UPLOAD YOUR kaggle.json FILE NOW ---


Saving kaggle.json to kaggle (1).json

--- Downloading Cocoa Data (zaldyjr) ---
Dataset URL: https://www.kaggle.com/datasets/zaldyjr/cacao-diseases
License(s): CC-BY-SA-4.0
 99% 1.00G/1.01G [00:16<00:00, 199MB/s]
100% 1.01G/1.01G [00:16<00:00, 65.0MB/s]

--- Downloading Maize Data (smaranjitghose) ---
Dataset URL: https://www.kaggle.com/datasets/smaranjitghose/corn-or-maize-leaf-disease-dataset
License(s): copyright-authors
 95% 152M/161M [00:00<00:00, 1.58GB/s]
100% 161M/161M [00:00<00:00, 1.48GB/s]

✅ Downloads Complete.


In [ ]:
import random
from pathlib import Path

# Define where we want the final data to live
base_dir = 'cropguard_dataset'
if os.path.exists(base_dir): shutil.rmtree(base_dir)
os.makedirs(f'{base_dir}/train')
os.makedirs(f'{base_dir}/val')

# --- Configuration: Map keywords to clean class names ---
# This tells the script what to look for in the unzipped folders
mappings = [
    # Cocoa Mappings
    {'keywords': ['Black', 'Pod'], 'class': 'cocoa_black_pod', 'source': 'temp_cocoa'},
    {'keywords': ['Healthy'],       'class': 'cocoa_healthy',   'source': 'temp_cocoa'},
    {'keywords': ['Pod', 'Borer'],  'class': 'cocoa_pod_borer', 'source': 'temp_cocoa'},

    # Maize Mappings
    {'keywords': ['Common', 'Rust'], 'class': 'maize_common_rust', 'source': 'temp_maize'},
    {'keywords': ['Gray', 'Leaf'],   'class': 'maize_gray_leaf_spot', 'source': 'temp_maize'},
    {'keywords': ['Blight'],         'class': 'maize_blight',      'source': 'temp_maize'},
    {'keywords': ['Healthy'],        'class': 'maize_healthy',     'source': 'temp_maize'}
]

print("--- Organizing Data ---")

for mapping in mappings:
    clean_name = mapping['class']
    keywords = mapping['keywords']
    source_root = mapping['source']

    # Find the actual folder path
    found_folder = None
    for root, dirs, files in os.walk(source_root):
        for d in dirs:
            # Check if all keywords are in the folder name (case insensitive)
            if all(k.lower() in d.lower() for k in keywords):
                found_folder = os.path.join(root, d)
                break
        if found_folder: break

    if found_folder:
        print(f"Found {clean_name} in: {found_folder}")

        # Get all images
        images = [x for x in os.listdir(found_folder) if x.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        # Split 80% Train / 20% Val
        split = int(len(images) * 0.8)
        train_imgs = images[:split]
        val_imgs = images[split:]

        # Create directories
        os.makedirs(f'{base_dir}/train/{clean_name}', exist_ok=True)
        os.makedirs(f'{base_dir}/val/{clean_name}', exist_ok=True)

        # Copy files
        for img in train_imgs:
            shutil.copy(os.path.join(found_folder, img), f'{base_dir}/train/{clean_name}/{img}')
        for img in val_imgs:
            shutil.copy(os.path.join(found_folder, img), f'{base_dir}/val/{clean_name}/{img}')
    else:
        print(f" WARNING: Could not find folder for {clean_name}")

print("\nDataset Organization Complete.")

--- Organizing Data ---
Found cocoa_black_pod in: temp_cocoa/cacao_diseases/cacao_photos/black_pod_rot
Found cocoa_healthy in: temp_cocoa/cacao_diseases/cacao_photos/healthy
Found cocoa_pod_borer in: temp_cocoa/cacao_diseases/cacao_photos/pod_borer
Found maize_common_rust in: temp_maize/data/Common_Rust
Found maize_gray_leaf_spot in: temp_maize/data/Gray_Leaf_Spot
Found maize_blight in: temp_maize/data/Blight
Found maize_healthy in: temp_maize/data/Healthy

✅ Dataset Organization Complete.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

print("Loading Data...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    'cropguard_dataset/train',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'cropguard_dataset/val',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

print(f"Classes: {train_ds.class_names}")

# Build Model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(train_ds.class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
print("\nStarting Training...")
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

print(f"\nFinal Accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")

Loading Data...
Found 6859 files belonging to 7 classes.
Found 1719 files belonging to 7 classes.
Classes: ['cocoa_black_pod', 'cocoa_healthy', 'cocoa_pod_borer', 'maize_blight', 'maize_common_rust', 'maize_gray_leaf_spot', 'maize_healthy']
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

Starting Training...
Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 97s 321ms/step - accuracy: 0.7712 - loss: 0.6946 - val_accuracy: 0.9156 - val_loss: 0.2617
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 37s 170ms/step - accuracy: 0.9072 - loss: 0.2657 - val_accuracy: 0.9290 - val_loss: 0.2178
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 36s 169ms/step - accuracy: 0.9184 - loss: 0.2217 - val_accuracy: 0.9337 - val_loss: 0.1968
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 40s 184ms/step - accuracy: 0.9238 - loss: 0.2002 - val_accuracy: 0.9337 - val_loss: 0.1840
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 38s 168ms/step - accuracy: 0.9308 - loss: 0.1876 - val_accuracy: 0.9389 - val_loss: 0.1755
Epoch 6/10
215/215 ━━━━━━━━━━━━

In [ ]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open('cropguard_model.tflite', 'wb') as f:
    f.write(tflite_model)

print(" Model Saved: cropguard_model.tflite")
print("Downloading to your computer...")

from google.colab import files
files.download('cropguard_model.tflite')

Saved artifact at '/tmp/tmp171v61g5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_238')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  136215431463376: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136215431463184: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  136215426169552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427434064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427433680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215426169936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427435024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427434640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427436752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136215427436944: TensorSpec(shape=(), dtype=tf.resource, name=

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>